In [ ]:
from typing import List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("juierror/text-to-sql-with-table-schema")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/text-to-sql-with-table-schema")

def prepare_input(question: str, table: List[str]):
    table_prefix = "table:"
    question_prefix = "question:"
    join_table = ",".join(table)
    inputs = f"{question_prefix} {question} {table_prefix} {join_table}"
    input_ids = tokenizer(inputs, max_length=700, return_tensors="pt").input_ids
    return input_ids

def inference(question: str, table: List[str]) -> str:
    input_data = prepare_input(question=question, table=table)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=700)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result

print(inference(question="get people name with age equal 25", table=["id", "name", "age"]))

In [ ]:
import pandas as pd
from sqlalchemy.engine import create_engine

In [ ]:
predicted_data_path = '/Users/macpro/Documents/GitHub/fuel-electric-hybrid-vehicle-ml/data/predicted-data/vehicle_data_with_clusters.csv'

df = pd.read_csv(predicted_data_path)

df.columns = df.columns.str.replace('.', '_')

# Replace the character '/' with '_per_' all entries
df.columns = df.columns.str.replace('/', '_per_')

# drop column hybrid_in_fuel	hybrid_in_electric	aggregate_levels	vehicle_type_cat
df = df.drop(['hybrid_in_fuel', 'hybrid_in_electric', 'aggregate_levels','transmission_','fuel_type'], axis=1)

In [ ]:
engine = create_engine("sqlite://")
df = pd.read_csv(predicted_data_path)

In [ ]:
df.columns

In [ ]:
print(inference(question="Show me hybrid car models", table=df.columns))

In [ ]:
%load_ext sql
%sql engine